In [16]:
#Import relevant libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import requests
from time import sleep
from random import randint
import pandas as pd
import numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [17]:
#Create a list of URLS to scrape
urls = ['https://www.imdb.com/title/tt2975590/?ref_=adv_li_i','https://www.imdb.com/title/tt2404233/?ref_=adv_li_i', 'https://www.imdb.com/title/tt2674426/?ref_=adv_li_i', 'https://www.imdb.com/title/tt2119532/?ref_=adv_li_i', 'https://www.imdb.com/title/tt1386697/?ref_=adv_li_i', 'https://www.imdb.com/title/tt4649466/?ref_=adv_li_i', 'https://www.imdb.com/title/tt1485796/?ref_=adv_li_i', 'https://www.imdb.com/title/tt0974015/?ref_=adv_li_i', 'https://www.imdb.com/title/tt1396484/?ref_=adv_li_i', 'https://www.imdb.com/title/tt5814060/?ref_=adv_li_i', 'https://www.imdb.com/title/tt3799232/?ref_=adv_li_i', 'https://www.imdb.com/title/tt6533240/?ref_=adv_li_i', 'https://www.imdb.com/title/tt6911608/?ref_=adv_li_i', 'https://www.imdb.com/title/tt3606756/?ref_=adv_li_i', 'https://www.imdb.com/title/tt5164214/?ref_=adv_li_i', 'https://www.imdb.com/title/tt1365519/?ref_=adv_li_i','https://www.imdb.com/title/tt2704998/?ref_=adv_li_i', 'https://www.imdb.com/title/tt1677720/?ref_=adv_li_i', 'https://www.imdb.com/title/tt3104988/?ref_=adv_li_i', 'https://www.imdb.com/title/tt6966692/?ref_=adv_li_i', 'https://www.imdb.com/title/tt5463162/?ref_=adv_li_i', 'https://www.imdb.com/title/tt4154756/?ref_=adv_li_i', 'https://www.imdb.com/title/tt2709692/?ref_=adv_li_i', 'https://www.imdb.com/title/tt0816692/?ref_=fn_al_tt_1', 'https://www.imdb.com/title/tt3315342/','https://www.imdb.com/title/tt1754316/','https://www.imdb.com/title/tt7394816/', 'https://www.imdb.com/title/tt1950235/', 'https://www.imdb.com/title/tt0137523/']

In [18]:
#Create a dataframe to store scraped data in (features as columns)
features = ['genre_0','genre_1','genre_2', 'genre_3','country','language','filming_locs','production_co','runtime',
'budget','gross_usa','release','director_0','director_1','rating','star_0','star_1','star_2','star_3','writer_0',
'writer_1','writer_2','review_count','title','open_week','cumulative']
movies = pd.DataFrame(data = np.empty((len(urls), len(features))),
                     columns= features)
movies[:] = np.nan

In [19]:
#loop through urls and scrape relevant features
for idx,url in enumerate(urls):
#     print(url)
    resp = requests.get(url)
    
#     session = requests.Session()
#     retry = Retry(connect=3, backoff_factor=0.5)
#     adapter = HTTPAdapter(max_retries=retry)
#     session.mount('http://', adapter)
#     session.mount('https://', adapter)

#     resp = session.get(url)
    
    #Pause the loop
    #sleep(randint(8,15))

    #Error if  status codes is not 200
    if resp.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))
    
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    #get genres
    genres_spec = soup.find_all('div', class_ = 'see-more inline canwrap')
    for i in range(len(genres_spec)):
        if 'Genres' in genres_spec[i].find('h4', class_ = 'inline').text:
            genres = soup.find_all('div', class_ = 'see-more inline canwrap')[i].find_all('a')
            movies.loc[idx,'genre_0'] = genres[0].text
            try:
                movies.loc[idx,'genre_1']= genres[1].text
                movies.loc[idx,'genre_2'] = genres[2].text
                movies.loc[idx,'genre_3'] = genres[3].text
            except:
                continue
    
    #Get country, language, filming location, production co, budget info variables
    other_specs = soup.find_all('div', class_ = 'txt-block')
    for i in range(len(other_specs)):
        if other_specs[i].find('h4', class_ = 'inline') is None:
            continue
        elif 'Country' in other_specs[i].find('h4', class_ = 'inline').text:
            movies.loc[idx,'country'] = other_specs[i].a.text
        elif 'Language' in other_specs[i].find('h4', class_ = 'inline').text:
            movies.loc[idx,'language'] = other_specs[i].a.text
        elif 'Filming Locations' in other_specs[i].find('h4', class_ = 'inline').text:
            movies.loc[idx,'filming_locs'] = other_specs[i].a.text
        elif 'Production Co' in other_specs[i].find('h4', class_ = 'inline').text:
            movies.loc[idx,'production_co'] =other_specs[i].a.text
        elif 'Runtime' in other_specs[i].find('h4', class_ = 'inline').text:
            movies.loc[idx, 'runtime'] = other_specs[i].time.text
        elif 'Budget' in other_specs[i].find('h4', class_ = 'inline').text:
            try:
                movies.loc[idx,'budget']= other_specs[i].text
            except: 
                continue
        elif 'Gross USA' in other_specs[i].find('h4', class_ = 'inline').text:
            try:
                movies.loc[idx,'gross_usa'] = other_specs[i].text
            except:
                continue
        elif 'Opening Weekend USA' in other_specs[i].find('h4', class_ = 'inline').text:
            try:
                movies.loc[idx,'open_week'] = other_specs[i].text
            except:
                continue
        elif 'Cumulative Worldwide Gross' in other_specs[i].find('h4', class_ = 'inline').text:
            try:
                movies.loc[idx,'cumulative'] = other_specs[i].text
            except:
                continue

    #get release date
    try:
        movies.loc[idx,'release'] = soup.find_all('div', class_ = 'subtext')[0].find_all('a', title = "See more release dates")[0].text
    except:
        continue
        
    #Get director, writer and stars    
    movie_containers = soup.find_all('div', class_ = 'credit_summary_item')
    for i in range(len(movie_containers)):
        if 'Director' in movie_containers[i].find('h4', class_ = 'inline').text:
            movies.loc[idx,'director_0'] = movie_containers[i].find_all('a')[0].text
            try:
                movies.loc[idx,'director_1']= movie_containers[i].find_all('a')[1].text
            except:
                continue
        if 'Writer' in movie_containers[i].find('h4', class_ = 'inline').text:
            movies.loc[idx,'writer_0']=movie_containers[i].find_all('a')[0].text
            try:
                movies.loc[idx,'writer_1']=movie_containers[i].find_all('a')[1].text
                movies.loc[idx,'writer_2']=movie_containers[i].find_all('a')[2].text
            except:
                continue

        if 'Star' in movie_containers[i].find('h4', class_ = 'inline').text:
            movies.loc[idx,'star_0']=movie_containers[i].find_all('a')[0].text
            try:
                movies.loc[idx,'star_1']=movie_containers[i].find_all('a')[1].text
                movies.loc[idx,'star_2']=movie_containers[i].find_all('a')[2].text
                movies.loc[idx,'star_3']=movie_containers[i].find_all('a')[3].text
            except:
                continue
        
    #Get raitng, review count and the title of the movie
    movies.loc[idx,'rating']=soup.find_all('span',{'itemprop':'ratingValue'})[0].text
    movies.loc[idx,'review_count']=soup.find_all('span',{'itemprop':'ratingCount'})[0].text
    movies.loc[idx,'title']=soup.find_all('title')[0].text

https://www.imdb.com/title/tt2975590/?ref_=adv_li_i
https://www.imdb.com/title/tt2404233/?ref_=adv_li_i
https://www.imdb.com/title/tt2674426/?ref_=adv_li_i
https://www.imdb.com/title/tt2119532/?ref_=adv_li_i
https://www.imdb.com/title/tt1386697/?ref_=adv_li_i
https://www.imdb.com/title/tt4649466/?ref_=adv_li_i
https://www.imdb.com/title/tt1485796/?ref_=adv_li_i
https://www.imdb.com/title/tt0974015/?ref_=adv_li_i
https://www.imdb.com/title/tt1396484/?ref_=adv_li_i
https://www.imdb.com/title/tt5814060/?ref_=adv_li_i
https://www.imdb.com/title/tt3799232/?ref_=adv_li_i
https://www.imdb.com/title/tt6533240/?ref_=adv_li_i
https://www.imdb.com/title/tt6911608/?ref_=adv_li_i
https://www.imdb.com/title/tt3606756/?ref_=adv_li_i
https://www.imdb.com/title/tt5164214/?ref_=adv_li_i
https://www.imdb.com/title/tt1365519/?ref_=adv_li_i
https://www.imdb.com/title/tt2704998/?ref_=adv_li_i
https://www.imdb.com/title/tt1677720/?ref_=adv_li_i
https://www.imdb.com/title/tt3104988/?ref_=adv_li_i
https://www.

In [20]:
movies

,genre_0,genre_1,genre_2,genre_3,country,language,filming_locs,production_co,runtime,budget,...,star_1,star_2,star_3,writer_0,writer_1,writer_2,review_count,title,open_week,cumulative
0,Action,Adventure,Sci-Fi,NaN,USA,English,"Detroit, Michigan, USA",Warner Bros.,151 min,"\nBudget:$250,000,000\n (estimated)\n",...,Henry Cavill,Amy Adams,See full cast & crew,Chris Terrio,David S. Goyer,5 more credits,"590,753",Batman v Superman: Dawn of Justice (2016) - IMDb,"\nOpening Weekend USA: $166,007,347,\n27 March...","\nCumulative Worldwide Gross: $873,634,919 ..."
1,Action,Adventure,Fantasy,NaN,USA,English,"Fox Studios, Moore Park, Sydney, New South Wal...",Pyramania,127 min,"\nBudget:$140,000,000\n (estimated)\n",...,Nikolaj Coster-Waldau,Gerard Butler,See full cast & crew,Matt Sazama,Burk Sharpless,NaN,"98,104",Gods of Egypt (2016) - IMDb,"\nOpening Weekend USA: $14,123,903,\n28 Februa...","\nCumulative Worldwide Gross: $150,680,864 ..."
2,Drama,Romance,NaN,NaN,UK,English,"Pembroke Castle, Pembrokeshire, Wales, UK",Metro-Goldwyn-Mayer (MGM),106 min,"\nBudget:$20,000,000\n (estimated)\n",...,Sam Claflin,Janet McTeer,See full cast & crew,Jojo Moyes,Jojo Moyes,NaN,"184,385",Me Before You (2016) - IMDb,"\nOpening Weekend USA: $18,723,269,\n5 June 2016","\nCumulative Worldwide Gross: $208,314,186 ..."
3,Biography,Drama,History,War,Australia,English,"Sydney, New South Wales, Australia",Cross Creek Pictures,139 min,"\nBudget:$40,000,000\n (estimated)\n",...,Sam Worthington,Luke Bracey,See full cast & crew,Robert Schenkkan,Andrew Knight,NaN,"385,052",Hacksaw Ridge (2016) - IMDb,"\nOpening Weekend USA: $15,190,758,\n6 Novembe...","\nCumulative Worldwide Gross: $175,302,354 ..."
4,Action,Adventure,Fantasy,Sci-Fi,USA,English,"Toronto, Ontario, Canada",Atlas Entertainment,123 min,"\nBudget:$175,000,000\n (estimated)\n",...,Jared Leto,Margot Robbie,See full cast & crew,David Ayer,NaN,NaN,"554,891",Suicide Squad (2016) - IMDb,"\nOpening Weekend USA: $133,682,248,\n7 August...","\nCumulative Worldwide Gross: $746,846,894 ..."
5,Action,Adventure,Comedy,NaN,UK,English,"Camberley, Surrey, England, UK",Twentieth Century Fox,141 min,"\nBudget:$104,000,000\n (estimated)\n",...,Colin Firth,Mark Strong,See full cast & crew,Jane Goldman,Matthew Vaughn,2 more credits,"243,534",Kingsman: The Golden Circle (2017) - IMDb,"\nOpening Weekend USA: $39,023,010,\n24 Septem...","\nCumulative Worldwide Gross: $410,902,662 ..."
6,Biography,Drama,Musical,NaN,USA,English,"1 E 78th St, New York City, New York, USA",Chernin Entertainment,105 min,"\nBudget:$84,000,000\n (estimated)\n",...,Michelle Williams,Zac Efron,See full cast & crew,Jenny Bicks,Bill Condon,1 more credit,"208,926",The Greatest Showman (2017) - IMDb,"\nOpening Weekend USA: $8,805,843,\n24 Decembe...","\nCumulative Worldwide Gross: $434,993,183 ..."
7,Action,Adventure,Fantasy,Sci-Fi,USA,English,"Leavesden Studios, Leavesden, Hertfordshire, E...",Warner Bros.,120 min,"\nBudget:$300,000,000\n (estimated)\n",...,Gal Gadot,Jason Momoa,See full cast & crew,Jerry Siegel,Joe Shuster,9 more credits,"346,641",Justice League (2017) - IMDb,"\nOpening Weekend USA: $93,842,239,\n19 Novemb...","\nCumulative Worldwide Gross: $657,924,295 ..."
8,Horror,NaN,NaN,NaN,USA,English,"Toronto, Ontario, Canada",New Line Cinema,135 min,"\nBudget:$35,000,000\n (estimated)\n",...,Jaeden Martell,Finn Wolfhard,See full cast & crew,Chase Palmer,Cary Joji Fukunaga,2 more credits,"410,047",It (2017) - IMDb,"\nOpening Weekend USA: $123,403,419,\n10 Septe...","\nCumulative Worldwide Gross: $700,449,318 ..."
9,Horror,Mystery,Thriller,NaN,USA,English,"Castel Film Studios, Bucharest, Romania",Atomic Monster,96 min,"\nBudget:$22,000,000\n (estimated)\n",...,Taissa Farmiga,Jonas Bloquet,See full cast & crew,Gary Dauberman,James Wan,1 more credit,"101,936",The Nun (2018) - IMDb,"\nOpening Weekend USA: $53,807,379,\n9 Septemb...","\nCumulative Worldwide Gross: $365,550,119 ..."


In [ ]:
#Export dataframe to csv
movies.to_csv('movies_scraping.csv',sep=',', index = None)